# Faster Style Transfer.

This notebook presents the **Faster Style Transfer** module usage steps and options. Emphasis was placed on the **inference/evaluation** part, and therefore on the actual images stylization. Model training guidelines are provided in the [GitHub repository](https://github.com/seloufian/Faster-Style-Transfer).

This project implementation in based on the paper "[A Learned Representation for Artistic Style](https://arxiv.org/abs/1610.07629)" realized in 2016 by Google Brain.

This style transfer technique, unlike the previous ones, uses a single Convolution Neural Network (CNN) trained on multiple styles, which offers a significant gain in storage memory and inference speed.

To speed up the styling process, the notebook **prioritizes the GPU** (Nvidia Cuda) usage, and if not available, the CPU is used instead.

Note that for it to run properly, the notebook **requires an internet connection** to load the illustration images, download model data and install required packages.

In [ ]:
# Temporary: Unzip the module code.
!unzip -q '/content/faster_style_transfer.zip'

## Project and Data Download.

This section groups and structures all project components required by the application: the style transfer module, required packages and data.

Clone the project's GitHub repository and set it as the current working directory.

In [ ]:
!git clone 'https://github.com/seloufian/Faster-Style-Transfer.git'

%cd 'faster_style_transfer/'

Install required Python packages.

In [ ]:
%pip install -r requirements.txt

Download the **artistic style images** on which the model was trained. These styles are used both by the model and for display/explanation purposes. Resized downloaded images can be found in this [Drive directory](https://drive.google.com/drive/folders/1CuKmsxpB7IH_1BdxJBYn0wx1oivJ9XqL), and the untouched original ones in this [Drive directory](https://drive.google.com/drive/folders/1o0iupQhGalGVA57mx7ZEb6vCrNLrp8Kc) (downloaded from [Google Arts](https://artsandculture.google.com/) using [Dezoomify](https://dezoomify.ophir.dev/)).

In [ ]:
!gdown -q --folder '1CuKmsxpB7IH_1BdxJBYn0wx1oivJ9XqL' -O 'data_style_transfer/'

Download the style transfer model best-results **checkpoint file** (25,026-th iteration), which contains the weights and other required metadata.
During training, checkpoints were saved every 1,000 iterations, they range from 86 to 28,036 and are available in this [Drive directory](https://drive.google.com/drive/folders/1s3z789wocNvRlPUeVt6S9mpxrinPp2oz).

In [ ]:
# Checkpoint filename: checkpoint_25026.tar
!gdown -q '1-ogTIuMNZ1XJj7rV_ckN80HrYTyYEBok' -O 'data_style_transfer/'

Download the **test images**, used for the stylization demo. Resized downloaded images can be found in this [Drive directory](https://drive.google.com/drive/folders/17s4zl1AFd4MP81_0pdzXJVWyQrVOpDN6), and the untouched original ones in this [Drive directory](https://drive.google.com/drive/folders/1_RWTd3BPVOENrEtn4hfH6WGiHCg2DPZG). In addition to these images, users can upload custom images for stylization.

In [ ]:
!gdown -q --folder '17s4zl1AFd4MP81_0pdzXJVWyQrVOpDN6' -O 'data_style_transfer/test/'

## Imports and Model Definition.

Import the model and required internal and external Python packages.

In [ ]:
from pathlib import Path

from PIL import Image

import torch
from torchvision import io
from torchvision.utils import save_image

from faster_style_transfer.process import build_model, predict, visualize_images
from faster_style_transfer.model import center_crop

Build the **style transfer model** and restore its weights from the checkpoint file.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = build_model(
    style_path='data_style_transfer/styles/',
    checkpoint_path='data_style_transfer/checkpoint_25026.tar',
    device=device
)

## Images Stylization.

The project's style transfer model is based on a single Convolution Neural Network (CNN) trained on multiple styles. For its inference, it requires input image(s) to be stylized, and other parameters depending on the inference type: **Single Image Stylization** and **Weighted Image Stylization**.

The list below contains the predefined test images. Although the model can process multiple images at once, in this demo, only one image is passed to the model. The test image can be modified just by uncommenting the chosen image path.
In addition, users can upload their own images in *JPG* or *PNG* format and replace the `CONTENT_IMAGE_PATH` variable.

In [ ]:
CONTENT_IMAGE_PATH = [
    # 'data_style_transfer/test/resized/martyrs_memorial.jpg',
    # 'data_style_transfer/test/resized/brad_pitt.jpg',
    # 'data_style_transfer/test/resized/golden_gate_bridge.jpg',
    # 'data_style_transfer/test/resized/women.jpg',
    # 'data_style_transfer/test/resized/hoover_tower.jpg',
    # 'data_style_transfer/test/resized/poznan_poland.jpg',
    'data_style_transfer/test/resized/tubingen.jpg',
]

To match the model's required input image size (**256 x 256**), uploaded images are automatically resized with **center cropping** to preserve their center content. The illustration below explains the *center-crop* process.

<img src="https://drive.google.com/uc?id=1_xSjyIIBk_0ZCDElOo5G2eMExfhUDn1M" alt="Image center-crop process" width="1024" height="auto">

### Single Image Stylization.

This stylization type applies a single style to an input image. The styles are identified by their index from 0 to 11 as the number of available styles which are shown in the illustration below.

<img src="https://drive.google.com/uc?id=1dVOv3RqFQvZ39Ljlz9Zt9_xx1rn0xruU" alt="Available Styles" width="1024" height="auto">

The chosen style can be changed just by uncommenting its corresponding line.

In [ ]:
SINGLE_STYLE = [
    # (0, 'annunciation_virgin_deal.jpg'),
    # (1, 'horses_on_the_seashore.jpg'),
    # (2, 'the_scream.jpg'),
    # (3, 'divan_japonais.jpg'),
    # (4, 'portrait_of_pablo_picasso.jpg'),
    # (5, 'three_fishing_boats.jpg'),
    # (6, 'the_trial.jpg'),
    # (7, 'great_wave_off_kanagawa.jpg'),
    # (8, 'tullia_ride_body_chariot.jpg'),
    # (9, 'head_of_a_clown.jpg'),
    # (10, 'bicentennial_print.jpg'),
    (11, 'the_starry_night.jpg'),
]

Then, the chosen image is stylized by the model. The output is shown below.

In [ ]:
# Styles are defined by their indexes.
style_index = SINGLE_STYLE[0][0]

# The model's prediction is a list of images.
out_image = predict(model,
    content_images_path=CONTENT_IMAGE_PATH,
    style_index=style_index
)

out_image[0]

### Weighted Image Stylization.

This stylization type allows a combination of multiple (at least 2) styles. It requires a list of style indexes and their weights (decimals, from 0.0 to 1.0) which must sum to 1, so that they form a [convex combination](https://en.wikipedia.org/wiki/Convex_combination).

Although the number of combined styles is not limited, in this demo, two types of combinations are considered: 2 and 4 styles.

#### Weighted 2-Styles Inference.

This weighted stylization performs style transfer by combining **two** styles. For this, from the list below, exactly two styles should be selected.

In [ ]:
WEIGHTED_TWO_STYLES = [
    # (0, 'annunciation_virgin_deal.jpg'),
    # (1, 1, 'horses_on_the_seashore.jpg'),
    (2, 'the_scream.jpg'),
    # (3, 'divan_japonais.jpg'),
    # (4, 'portrait_of_pablo_picasso.jpg'),
    # (5, 'three_fishing_boats.jpg'),
    # (6, 'the_trial.jpg'),
    # (7, 'great_wave_off_kanagawa.jpg'),
    # (8, 'tullia_ride_body_chariot.jpg'),
    # (9, 'head_of_a_clown.jpg'),
    # (10, 'bicentennial_print.jpg'),
    (11, 'the_starry_night.jpg'),
]

In this demo, the combination weights are set to form a smooth transition from one style to another. The illustration below shows the sub-colors between two colors (the sides: left and right) by applying the combination weights.

<img src="https://drive.google.com/uc?id=15oE6GSV5MflSACatRMi8XQRXO46wDEgV" alt="2-Styles inference weights" width="1024" height="auto">

The code below defines the weights, applies them to the previously defined test image, and displays the result in a grid.

In [ ]:
WEIGHT_STEPS = [0, 0.25, 0.5, 0.75, 1]

In [ ]:
assert len(WEIGHTED_TWO_STYLES) == 2, 'Exactly TWO styles must be selected!'

style_index = [style[0] for style in WEIGHTED_TWO_STYLES]

In [ ]:
sides, rows = [], []

for weight_step in WEIGHT_STEPS:
    weights = [weight_step, 1-weight_step]

    out_image = predict(model,
        content_images_path=CONTENT_IMAGE_PATH,
        style_index=style_index,
        weights=weights
    )

    if 0 in weights:
        # The current image is one of the sides (left/right).
        sides.append(out_image[0])
    else:
        # The current image is one of the in-between sub-columns.
        rows.append(out_image[0])

In [ ]:
visualize_images(sides, [rows])

#### Weighted 4-Styles Inference.

Same as the previous stylization except now **four** styles are combined. From the list below, exactly four styles should be chosen.

In [ ]:
WEIGHTED_FOUR_STYLES = [
    # (0, 'annunciation_virgin_deal.jpg'),
    # (1, 1, 'horses_on_the_seashore.jpg'),
    (2, 'the_scream.jpg'),
    (3, 'divan_japonais.jpg'),
    (4, 'portrait_of_pablo_picasso.jpg'),
    # (5, 'three_fishing_boats.jpg'),
    # (6, 'the_trial.jpg'),
    # (7, 'great_wave_off_kanagawa.jpg'),
    # (8, 'tullia_ride_body_chariot.jpg'),
    # (9, 'head_of_a_clown.jpg'),
    # (10, 'bicentennial_print.jpg'),
    (11, 'the_starry_night.jpg'),
]

The defined combination weights form a smooth transition between the four styles. The illustration below shows the sub-colors between four colors (the 4 sides: upper-left, upper-right, lower-left, and lower-right) resulting from the weights application.

<img src="https://drive.google.com/uc?id=17iDXTq20u-2J3F-FcAcM1WJsJZoTxLlk" alt="4-Styles inference weights" width="1024" height="auto">

In [ ]:
WEIGHT_4_STYLES_STEPS = [
    [[1, 0, 0, 0],
     [0.75, 0.25, 0, 0],
     [0.5, 0.5, 0, 0],
     [0.25, 0.75, 0, 0],
     [0, 1, 0, 0]],
    [[0.75, 0, 0.25, 0],
     [0.5625, 0.1875, 0.1875, 0.0625],
     [0.375, 0.375, 0.125, 0.125],
     [0.1875, 0.5625, 0.0625, 0.1875],
     [0, 0.75, 0, 0.25]],
    [[0.5, 0, 0.5, 0],
     [0.375, 0.125, 0.375, 0.125],
     [0.25, 0.25, 0.25, 0.25],
     [0.125, 0.375, 0.125, 0.375],
     [0, 0.5, 0, 0.5]],
    [[0.25, 0, 0.75, 0],
     [0.1875, 0.0625, 0.5625, 0.1875],
     [0.125, 0.125, 0.375, 0.375],
     [0.0625, 0.1875, 0.1875, 0.5625],
     [0, 0.25, 0, 0.75]],
    [[0, 0, 1, 0],
     [0, 0, 0.75, 0.25],
     [0, 0, 0.5, 0.5],
     [0, 0, 0.25, 0.75],
     [0, 0, 0, 1]],
]

The following code defines the weights, applies them to the previously defined test image and displays the result in a grid.

In [ ]:
assert len(WEIGHTED_FOUR_STYLES) == 4, 'Exactly FOUR styles must be selected!'

style_index = [style[0] for style in WEIGHTED_FOUR_STYLES]

In [ ]:
STYLES_DIR = Path('data_style_transfer/styles/')

sides = []

# Selected style images are loaded and resized to "256 x 256"
# (like the model output), and added to the 4 sides of the grid.
for _, style_name in WEIGHTED_FOUR_STYLES:
    curr_style = Image.open(STYLES_DIR / style_name)
    curr_style = curr_style.resize((256, 256))

    sides.append(curr_style)

In [ ]:
rows = []

# Loop over the 5 rows of the grid.
for row_weights in WEIGHT_4_STYLES_STEPS:
    row_images = []

    # Loop over the 5 columns of the row.
    # Each column is a combination of:
    # - The upper and the lower rows (relative to the current row).
    # - The columns on the left and on the right (relative the current column).
    for weights in row_weights:
        out_image = predict(model,
            content_images_path=CONTENT_IMAGE_PATH,
            style_index=style_index,
            weights=weights
        )

        row_images.append(out_image[0])

    rows.append(row_images)

In [ ]:
visualize_images(sides, rows)